# Домашнее задание №4
# Тематическое моделирование. EM-алгоритм

## Задание

Объединить в одну выборку положительные и отрицательные твиты.

Провести исследование и выявить тематики, о которых говорят в твитах.

Сделать визуализацию кластеров тематик.

Проинтерпритировать получившиеся тематики

## Подготовка

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=6f549d52b59347eb54174c2841acc6e2035cc74e890d0ca76fc863dfcf323865
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=55ca72418a77c8193ffbc44fba77f0c1d4eaaf9fdb3b5d426c16609ed2a303d6
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=cbeb64826e45360162dfb96549f500c38f3cfeff00e975d552bbcc5fd834c2ac
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [ ]:
# Импорт библиотек
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
import pymorphy2
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Загрузка данных

In [ ]:
# Подключаем диск
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Устанавливаем путь к папке проекта
ROOT_PATH = '/gdrive/MyDrive/Colab Notebooks/NLP/Захаров Роман/4_Тематическое_моделирование'

In [ ]:
# Считаем данные и объединим в один датасет
positive = pd.read_csv(Path(ROOT_PATH).joinpath('positive.csv'), sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv(Path(ROOT_PATH).joinpath('negative.csv'), sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative], ignore_index=True)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [ ]:
# Вывод информации о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


## Предобработка данных

In [ ]:
words_regex = re.compile('\w+') # Компиляция объекта (1 или более букв)

# Список токенов из букв и длины >3
def find_words(text, regex = words_regex):
    tokens = regex.findall(text) # ищем все слова в тексте --> получаем список токенов
    return [w for w in tokens if w.isalpha() and len(w)>= 4] # берем токеты из списка токетов, если они состоят только из букв и имеют длину >3

morph = pymorphy2.MorphAnalyzer() # Морфологический анализ
stopwords_list = stopwords.words('russian') # Список стоп-слов

# Список нормальных форм токенов, убираем стоп-слова и токены с небуквами 
def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] #список нормальных форм для всех токенов
    return [w for w in lemmas if not w in stopwords and w.isalpha()] #удаляем из списка нормальных форм стоп-слова и токены с небуквой(ми)

# Очищаем токены от нецензурной лексики и ненужныъ слов
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
<ipython-input-15-d55989d9e4f1>:1: DeprecationWarning: invalid escape sequence \w
  words_regex = re.compile('\w+') # Компиляция объекта (1 или более букв)
/usr/local/lib/python3.8/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.8/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.8/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = ins

In [ ]:
# Предобработка текста
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, немного, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


## Модель LDA

In [ ]:
# Словарь уникальных токенов

dictionary = corpora.Dictionary(df['text']) # создаем словарь между токенами и их индексами (на вход список списков)

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 

dictionary.save('tweet.dict')

len(dictionary.keys()) # длина словаря из токенов

/usr/local/lib/python3.8/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


8172

In [ ]:
# Векторизируем твиты
corpus = [dictionary.doc2bow(text) for text in df['text']] # для каждого списка токенов возвращает кортеж (индекс токена, количество встречаний токена в твите)
corpora.MmCorpus.serialize('tweet.model', corpus) # выполняем итерацию по корпусу (записываем смещение байта каждого твита). Полученную структуру индексов сохраняем в файл


/usr/local/lib/python3.8/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.8/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [ ]:
# Обучаем модель
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sc

In [ ]:
# Определяем вероятностные темы
lda.show_topics(num_topics=3, num_words=20, formatted=True)

[(0,
  '0.014*"идти" + 0.013*"друг" + 0.012*"настроение" + 0.011*"голова" + 0.009*"скоро" + 0.009*"найти" + 0.009*"экзамен" + 0.009*"сегодня" + 0.009*"каникулы" + 0.007*"первый" + 0.007*"снег" + 0.006*"фотка" + 0.006*"парень" + 0.006*"самый" + 0.006*"чувство" + 0.006*"глаз" + 0.005*"нога" + 0.005*"момент" + 0.005*"рука" + 0.005*"ужас"'),
 (1,
  '0.028*"завтра" + 0.024*"день" + 0.023*"сегодня" + 0.021*"дом" + 0.017*"хороший" + 0.017*"школа" + 0.015*"человек" + 0.014*"жизнь" + 0.012*"утро" + 0.011*"жаль" + 0.011*"время" + 0.010*"час" + 0.009*"никто" + 0.009*"урок" + 0.008*"прийти" + 0.007*"работа" + 0.007*"неделя" + 0.007*"последний" + 0.007*"самый" + 0.007*"улица"'),
 (2,
  '0.017*"новый" + 0.013*"больш" + 0.013*"мама" + 0.011*"год" + 0.011*"любимый" + 0.010*"ночь" + 0.010*"день" + 0.009*"каждый" + 0.009*"человек" + 0.008*"телефон" + 0.008*"втор" + 0.008*"казаться" + 0.008*"нравиться" + 0.007*"бояться" + 0.006*"твиттер" + 0.006*"пока" + 0.006*"минута" + 0.006*"учёба" + 0.006*"деньга" + 

In [ ]:
# Оценка качества модели
print(lda.log_perplexity(corpus))
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sc

-8.00928896429967


Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sc

Персплексия:  0.00033236095510574706


/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

## Визуализация предсказаний модели

In [ ]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


## Заключение

Можно сформулировать следующие примерные названия тем твитов:

**1.** Учеба / работа

**2.** Новый год

**3.** Сессия / экзамены